# 4-1. Construct AI Demand Exposure Score - Preparatory Step (Test LLMs)

Job posting data is provided by: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings

**Author:** Yu Kyung Koh

**Last Updated:** 2025/06/14

---

### General Goals:
* Construct AI Demand Exposure Score, which measures how susceptible job tasks listed in new postings are to augmentation or replacement by AI
* I am going to follow the approach in 2025 Revelio Labs report "AI at Work: The State of AI Adoption in 2025"

**Steps to construct the AI demand exposure score:**
* **PART #1:** Extract tasks from job postings **-> ‼️FOCUS OF THIS CODE‼️**
   * Extract clean and relevant task-like phrases (e.g., "analyze data", "prepare reports")
   * Find a way to ignore the boilerplate sections (e.g., DEI statements, benefits).
     * => Going to use LLM to extract action/task-oriented lines
* **PART #2:** Map tasks to a taxonomy (optional but strongly recommended)
    * Standardize the language of tasks across postings (like Revelio’s “activities taxonomy”)
    * Alternatively, cluster similar task phrases using embeddings + k-means or sentence similarity.
* **PART #3:** Score Task Susceptibility to AI (Using GPT or LLM)
  * Goal: Determine whether a task can be augmented or automated by AI.
  * Use GPT-4 or similar model with a prompt like:
    * "Can the following task be automated or enhanced using AI tools such as ChatGPT, image recognition, or robotic process automation? Respond 'Yes' or 'No' and briefly justify."
    * Input: one task per call (or a batch, if needed).
    * Response: Boolean + rationale (you can discard rationale for speed).
  * Store the binary AI-susceptibility flag for each task.
* **PART #4:** Compute the AI Demand Exposure Score

---
### In this code:
* I test a few LLMs on a small sample of job postings for task extraction.
* This will help me choose the most suitable LLM for extracting tasks from the full dataset, which would otherwise be time-consuming.
* **Heads-up:**
  * I learned that **mistral model via Ollama** outperforms other executable models (e.g. flat-t5) on my Macbook.
  * This will be discussed in detail in SECTION 5. 

---

## SECTION 1: Ways to extract task descriptions from unstructured, messy job postings

There are three main ways to extract task description from unstructured job postings. 

#### Option 1: ML-based sentence classification
* Train a binary classifier to label each sentence in a job posting as: **task-related** (1) or **not task-related** (0)
* How to implement
  1. Split job postings into sentences.
  2. Label a small training set (manually tag ~500–1,000 sentences).
  3. Fine-tune a transformer like distilbert-base-uncased or roberta-base on this sentence classification task.
  4. Apply model to reset the rest of the job postings.
* **✅ Pros**: High flexibility and adaptability, Handles varying structures well
* **❌ Cons**: Requires some labeled data and compute,  Harder to debug than rules
  
#### Option 2: LLM-based zero-shot or few-shot classification    **-> ‼️BEST APPROACH‼️**
* Use GPT (or other LLM) to label sentences or extract task descriptions directly.
    * 🔨 Prompt Example:
    "Below is a job posting. Extract only the specific tasks that an employee would be expected to perform in this role. Ignore company values, benefits, and general statements."
    * Then pass the full job posting, or chunks of it, and parse the output.
* **✅ Pros**: No training required, Often very accurate on diverse formats
* **❌ Cons**: Slower, Costs may add up if processing many postings, May require post-cleaning to standardize output

#### Option 3: Heuristic-based rules (regex + patterns)
* I can combine the following rules:
    1. Bullet Points or Imperative Sentences
        * Regex: r"^\s*[-•*]\s+.+"
        * Captures bullet-style task listings.
    2. Verb-First Sentences
        * Use POS tagging to retain sentences that begin with strong verbs (VB, VBP, VBZ).
    3. Section-Based Extraction
        * Try to identify "Job Responsibilities", "What You’ll Do", "Duties" sections using regex or keyword matching, and extract bullet points or paragraphs under them.

* **❌ Cons**: Not straightforward with the data I have, which may not have bullet points (Rule 1) or clear section titles (Rule 3)

---

In this code, I will focus on  **Option 2: LLM-based zero-shot or few-shot classification**, as this seems most promising. 

## SECTION 2: Bring in the job posting data 

Here, I am going to focus on the smaller set of job posting data cleaned from code 2-1. 

This data only contains job posting for few job categories, including data-related jobs, consultants, software enginners, etc. 

In [4]:
import pandas as pd
import os
import re
import joblib
from tqdm import tqdm
import multiprocessing 
from joblib import Parallel, delayed

import nltk
from nltk.corpus import stopwords
#from rapidfuzz import process, fuzz

In [5]:
## Import cleaned data from Code 2-1
cleandatadir = '/Users/yukyungkoh/Desktop/1_Post-PhD/7_Python-projects/2_practice-NLP_job-posting_NEW/2_data/cleaned_data'
jobdata = os.path.join(cleandatadir, '1_job-posting_jobs-categorized_df.pkl')
jobs_df = pd.read_pickle(jobdata, 'zip')
#jobs_df = joblib.load(jobdata)

## Check how many job postings are in this data 
len(jobs_df)

29724

In [6]:
jobs_df.head()

,job_id,company_name,title,work_type,normalized_salary,combined_desc,job_category
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,FULL_TIME,38480.0,Job descriptionA leading real estate firm in N...,Marketing
2,10998357,The National Exemplar,Assitant Restaurant Manager,FULL_TIME,55000.0,The National Exemplar is accepting application...,Other Manager
12,56482768,NaN,Appalachian Highlands Women's Business Center,FULL_TIME,NaN,FULL JOB DESCRIPTION – PROGRAM DIRECTOR Appala...,Business/Finance Job
14,69333422,Staffing Theory,Senior Product Marketing Manager,FULL_TIME,NaN,A leading pharmaceutical company committed to ...,Marketing
18,111513530,United Methodists of Greater New Jersey,"Content Writer, Communications",FULL_TIME,NaN,"Application opening date: April 24, 2024\nTitl...",Marketing


---
## SECTION 3: Extract tasks using "google/flan-t5-large" (Try with 10 job postings first)

### Note 1: 
* There are multiple ways to implement **Option 2: LLM-based zero-shot or few-shot classification**
  1. **Option 2A:** **GPT (via openai package)**  — Paid, but easy to use with strong performance and long context windows. Requires internet access
  2. **Option 2B:** **Hugging Face Transformers (e.g., flan-t5-large)** - Free and open-source; runs locally but may be slower or memory-intensive on personal machines
  3. **Option 2C:** **Ollama (Local quantized models like Mistral or LLaMA)** - Free, runs entirely offline, optimized for Mac (M1/M2); Slower than cloud models but efficient and private. 
* Since I prefer a free option, I will stick to **Option 2B** in this section. In the next section (Section 4), I will use Option 2C: Ollama and compare its performance with Option 2B. 

### Note 2:
* With  Hugging Face Open-Source LLM, there are multiple models that I can use, which differ in their performances and MacBook-friendliness.
* Below is a list of models I can use with `pipeline("text-generation", model=...)` for extracting task descriptions from job postings.

     1. ✅ Tiny / CPU-Friendly Models (Great for Testing on MacBook)

| Model | Description | RAM Needed |
|-------|-------------|------------|
| `sshleifer/tiny-gpt2` | Toy model for debugging | ~200MB |
| `EleutherAI/gpt-neo-125M` | Small GPT-style model | ~1GB |
| `tiiuae/falcon-rw-1b` | Reasoning-capable small model | ~2.5GB |
| `EleutherAI/gpt-neo-1.3B` | Higher-quality small model | ~4GB |


    2. ✅ Instruction-Tuned Small Models (Mac-Friendly + Follows Prompts Better)

| Model | Description | RAM Needed |
|-------|-------------|------------|
| `google/flan-t5-base` | Instruction-tuned T5 | ~2GB |
| `google/flan-t5-large` | Larger, better output | ~5–6GB |
| `declare-lab/flan-alpaca-base` | Alpaca-style model | ~4–5GB |

    3. ⚠️ Large Instruction-Following LLMs (Require GPU or 32GB+ RAM)

| Model | Description | RAM Needed |
|-------|-------------|------------|
| `HuggingFaceH4/zephyr-7b-beta` | Zephyr 7B instruction model | ~14–16GB |
| `mistralai/Mistral-7B-Instruct-v0.1` | Strong 7B LLM | ~16GB |
| `meta-llama/Llama-3-8B-Instruct` | State-of-the-art LLaMA 3 | ~18GB |
        * Use these only in **Google Colab with GPU** or on machines with large RAM.

In [8]:
## ---------------------------------------------
## STEP 1: Load LLM model for task extraction 
## ---------------------------------------------
from transformers import pipeline

task_extractor = pipeline(
    "text2text-generation",
    # model="HuggingFaceH4/zephyr-7b-beta", ## => Takes too long to load. May not work with my macbook. 
    model="google/flan-t5-large", 
    framework="pt",  # 👈 this avoids TensorFlow completely
    max_new_tokens=300
)

Device set to use mps:0


In [9]:
## ---------------------------------------------
## STEP 2: Extract tasks for the first 10 job postings 
## ---------------------------------------------

import time
start_time = time.time()  # Start the timer

# Limit to the first 10 job postings
sample_jobs = jobs_df["combined_desc"].head(10)

# Define a prompt template
def format_prompt(text):
    return f"""Extract the specific job tasks or responsibilities from the following job posting. 
                Do NOT include phrases about qualifications, skills, company info, contact info, or benefits.
                Only list tasks as bullet points.\n\n{text}"""

# Apply the task extractor
extracted_tasks = []

for idx, posting in enumerate(sample_jobs): ## Remember that enumerate() returns a pair (index, value) 
    prompt = format_prompt(posting)
    try:
        result = task_extractor(prompt)[0]["generated_text"]
    except Exception as e:
        result = f"Error extracting tasks: {e}"
    extracted_tasks.append(result)


results_df = pd.DataFrame({
    "job_posting_text": sample_jobs.values,
    "extracted_tasks_flant5": extracted_tasks
})

end_time = time.time()  # Stop the timer

# Show the time it took for task extraction for the first 10 job postings 
from datetime import timedelta
elapsed_time = end_time - start_time
formatted_time = str(timedelta(seconds=int(elapsed_time)))
print(f"\nTotal time taken: {formatted_time} (hh:mm:ss)")


Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors



Total time taken: 0:07:02 (hh:mm:ss)


In [10]:
# Show results
from IPython.display import display
display(results_df)

,job_posting_text,extracted_tasks_flant5
0,Job descriptionA leading real estate firm in N...,Administrative Marketing Coordinator with some...
1,The National Exemplar is accepting application...,The National Exemplar is accepting application...
2,FULL JOB DESCRIPTION – PROGRAM DIRECTOR Appala...,APPALACHIAN HIGHLANDS WOMEN’S BUSINESS CENTER ...
3,A leading pharmaceutical company committed to ...,Develop and implement integrated marketing pla...
4,"Application opening date: April 24, 2024\nTitl...",The Content Writer develops and disseminates i...
5,"Education Bachelor's degree in software, math,...","Analytical skills, group work, knowledge of in..."
6,Job Description:GOYT is seeking a skilled and ...,GOYT is seeking a skilled and motivated Remote...
7,About Revesco Properties:Revesco Properties is...,The ideal candidate has experience in developi...
8,SUMMARY:Manages operation and supervises all d...,Manages operation and supervises all departmen...
9,Gordon Partners (www.gordonpartners.com) is se...,Position will work directly with both the dire...


In [11]:
results_df["job_posting_text"][8]

'SUMMARY:Manages operation and supervises all departmental distribution/clinical and educational activities; plans, controls, coordinates and measures the work of the department.ESSENTIAL FUNCTIONS:Manages staff; interviews, hires and trains; evaluates employee performance; deals with performance problems as appropriate; delegates work assignments effectively.Assists in managing department budget.Manages Pharmacy operations and coordinates functions with the needs of other departments. Oversees and manages drug purchases, information and review for drug interactions.Benchmarks pharmacy operations with local/regional and national solutions.Critically reviews the medical literature; collates and summarizes studies and makes recommendations to the appropriate party.Networks with hospital departments, takes input and in conjunction with Administration and Pharmacy Department to develop projects, and monitors their progress to completion.Monitors pharmacy payment methodologies and pharmacy 

In [12]:
results_df["extracted_tasks_flant5"][8]

'Manages operation and supervises all departmental distribution/clinical and educational activities; plans, controls, coordinates and measures the work of the department. Oversees and manages drug purchases, information and review for drug interactions.Benchmarks pharmacy operations with local/regional and national solutions. Reviews the medical literature; collates and summarizes studies and makes recommendations to the appropriate party.Networks with hospital departments, takes input and in conjunction with Administration and Pharmacy Department to develop projects, and monitors their progress to completion. Monitors pharmacy payment methodologies and pharmacy systems to ensure accuracy and understanding by staff. Assimilates pharmacy/hospital projects into presentations that can be conveyed in an interesting and positive manner on the hospital’s behalf. Adheres to TMC organizational and department-specific safety, confidentiality, values policies and standards.Performs related dutie

In [13]:
## ---------------------------------------------
## STEP 3: Extract tasks for the first 10 job postings -- Sliding window chunking approach
## ---------------------------------------------

---
## SECTION 4: Extract tasks using the Mistral model via Ollama (Try with 10 job postings first)

Another way to use LLM is through Ollama. 
* Note that Ollama is a **local model runner** with a command-line tool and a Python API
    *  This differs from Hugging Face Transformer I used in SECTION 3 -- which is a **Python library**. 
    * They both let me run LLMs, but they differ in **design, purpose, and performance**
* Specifically, I am going to use the **mistral** large language model via Ollama

### ✅ Summary Table: Hugging Face Transformers vs Ollama

| Feature                        | 🤖 Hugging Face Transformers                    | 🐙 Ollama                                    |
|-------------------------------|--------------------------------------------------|----------------------------------------------|
| **What it is**                | Python library for loading any model            | Tool for running local quantized models      |
| **How you use it**            | `transformers` library in Python                | Terminal (`ollama run`) or Python API        |
| **Model format**              | Hugging Face format (`.bin`, `.safetensors`)    | GGUF format (quantized models)               |
| **Performance on M1**         | ⚠️ Slower (CPU-only, full precision)            | ✅ Fast (Metal + quantized models)           |
| **Ease of setup**             | Medium (need PyTorch, model, RAM)               | ✅ Easy (one-line install)                    |
| **RAM usage**                 | High (8–16GB for large models)                  | Low (2–6GB with quantization)                |
| **Offline use**               | ✅ Yes (once downloaded)                         | ✅ Yes (fully offline after setup)            |
| **Supports fine-tuning**      | ✅ Yes (train/fine-tune models)                 | ❌ No (inference only)                        |
| **Model availability**        | Thousands on Hugging Face Hub                   | Dozens curated (e.g., Mistral, LLaMA 3)      |
| **Interface style**           | Task-based or causal (generation, QA)           | Chat-style (`messages=[...]`)                |
| **Good for batch processing** | ✅ Yes (Python-native)                          | ⚠️ Limited, one call at a time               |
| **Best suited for**           | Researchers, developers needing control         | Easy local inference on laptops              |



### 🧠 Conceptual Differences

| Area              | Hugging Face Transformers                                | Ollama                                         |
|-------------------|-----------------------------------------------------------|------------------------------------------------|
| **Flexibility**   | Full control over models, tokenizers, architecture        | Limited to prebuilt models in GGUF             |
| **Ease of use**   | More complex (need to load + configure models)            | One command: `ollama run mistral`              |
| **Customization** | Train, modify, or stack models                            | Inference only (no fine-tuning)                |
| **Execution**     | Heavy backend (PyTorch, TensorFlow)                       | Lightweight Metal-accelerated C++ engine       |
| **API style**     | Versatile: `pipeline(...)`, `model.generate(...)`         | Chat-style: `chat(model="mistral", ...)`       |


In [15]:
from ollama import chat

### Note: 
#   - Before running this, 
#     I need to type "ollama run mistral" in the terminal

### Initialize list for storing results
extracted_tasks_mistral = []

### Loop through job postings in existing results_df
for desc in tqdm(results_df["job_posting_text"]):
    prompt = f"""Extract the main job **tasks** from the following job posting. 
List distinct, clear bullet points. Only include tasks, not benefits or requirements.

Job posting:
\"\"\"{desc}\"\"\"
"""
    response = chat(model='mistral', messages=[
        {'role': 'user', 'content': prompt}
    ])
    
    extracted = response['message']['content']
    extracted_tasks_mistral.append(extracted)

### Add new column to results_df
results_df["extracted_tasks_mistral"] = extracted_tasks_mistral

  0%|                                                    | 0/10 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████████████| 10/10 [03:49<00:00, 23.00s/it]


---
## SECTION 5: Check performance: flan-t5-large (via HuggingFace) vs. Mistral (via Ollama)

* Below, I am going to examine the performance of the flan-t5-large model (SECTION 3) and the mistral model (SECTION 4) by examining the extracted tasks from each LLM. 

In [17]:
display(results_df)

,job_posting_text,extracted_tasks_flant5,extracted_tasks_mistral
0,Job descriptionA leading real estate firm in N...,Administrative Marketing Coordinator with some...,- Receive and organize marketing requests fro...
1,The National Exemplar is accepting application...,The National Exemplar is accepting application...,- Deliver high quality customer service\n- Ma...
2,FULL JOB DESCRIPTION – PROGRAM DIRECTOR Appala...,APPALACHIAN HIGHLANDS WOMEN’S BUSINESS CENTER ...,- Provide strategic direction for training pr...
3,A leading pharmaceutical company committed to ...,Develop and implement integrated marketing pla...,- Develop integrated marketing plans for phar...
4,"Application opening date: April 24, 2024\nTitl...",The Content Writer develops and disseminates i...,- Develop engaging content across various cha...
5,"Education Bachelor's degree in software, math,...","Analytical skills, group work, knowledge of in...","1. Possess a Bachelor's degree in software, m..."
6,Job Description:GOYT is seeking a skilled and ...,GOYT is seeking a skilled and motivated Remote...,* Develop and maintain high-quality PHP code ...
7,About Revesco Properties:Revesco Properties is...,The ideal candidate has experience in developi...,* Develop and execute marketing plans and str...
8,SUMMARY:Manages operation and supervises all d...,Manages operation and supervises all departmen...,- Supervise all departmental distribution/cli...
9,Gordon Partners (www.gordonpartners.com) is se...,Position will work directly with both the dire...,- Maintain and grow the portfolio of owned an...


In [18]:
## Examine job posting 8 -- which has reasonable length 
results_df["job_posting_text"][8]

'SUMMARY:Manages operation and supervises all departmental distribution/clinical and educational activities; plans, controls, coordinates and measures the work of the department.ESSENTIAL FUNCTIONS:Manages staff; interviews, hires and trains; evaluates employee performance; deals with performance problems as appropriate; delegates work assignments effectively.Assists in managing department budget.Manages Pharmacy operations and coordinates functions with the needs of other departments. Oversees and manages drug purchases, information and review for drug interactions.Benchmarks pharmacy operations with local/regional and national solutions.Critically reviews the medical literature; collates and summarizes studies and makes recommendations to the appropriate party.Networks with hospital departments, takes input and in conjunction with Administration and Pharmacy Department to develop projects, and monitors their progress to completion.Monitors pharmacy payment methodologies and pharmacy 

In [19]:
results_df["extracted_tasks_flant5"][8]

'Manages operation and supervises all departmental distribution/clinical and educational activities; plans, controls, coordinates and measures the work of the department. Oversees and manages drug purchases, information and review for drug interactions.Benchmarks pharmacy operations with local/regional and national solutions. Reviews the medical literature; collates and summarizes studies and makes recommendations to the appropriate party.Networks with hospital departments, takes input and in conjunction with Administration and Pharmacy Department to develop projects, and monitors their progress to completion. Monitors pharmacy payment methodologies and pharmacy systems to ensure accuracy and understanding by staff. Assimilates pharmacy/hospital projects into presentations that can be conveyed in an interesting and positive manner on the hospital’s behalf. Adheres to TMC organizational and department-specific safety, confidentiality, values policies and standards.Performs related dutie

In [20]:
results_df["extracted_tasks_mistral"][8]

' - Supervise all departmental distribution/clinical and educational activities\n- Plan, control, coordinate, and measure the work of the department\n- Manage staff (interview, hire, train)\n- Evaluate employee performance and deal with performance problems as appropriate\n- Delegate work assignments effectively\n- Assist in managing department budget\n- Manage Pharmacy operations and coordinate functions with other departments\n- Oversees drug purchases, information, and review for drug interactions\n- Benchmark pharmacy operations with local/regional and national solutions\n- Critically review the medical literature, collate and summarize studies, make recommendations to appropriate parties\n- Network with hospital departments and develop projects in conjunction with Administration and Pharmacy Department\n- Monitor progress of pharmacy projects to completion\n- Monitor pharmacy payment methodologies and pharmacy systems to ensure accuracy and understanding by staff\n- Assimilate pha

In [21]:
## Examine job posting 2 -- which is very long 
results_df["job_posting_text"][2]

"FULL JOB DESCRIPTION – PROGRAM DIRECTOR Appalachian Highlands Women’s Business Center Kingsport, TennesseeDepartment: Kingsport Office of Small Business Development & Entrepreneurship (KOSBE)Reports to: Chief Business Development Officer, Kingsport Chamber - KOSBEDirect Reports: AppH-WBC StaffType of Position: DirectorWork Schedule: Full-Time (40 hours per week). Exempt Status: ExemptLocation: This is not a remote position, this is in office in Kingsport, Tenn. AppH-WBC is located in Kingsport, Tennessee, under the Kingsport Chamber and within the Kingsport Office of Small Business Development & Entrepreneurship (KOSBE). The office is within walking distance to downtown living, dining and shopping.ABOUT KOSBE:In 2004, the Kingsport Chamber and City of Kingsport jointly formed the Kingsport Office of Small Business Development & Entrepreneurship (KOSBE), to specifically nurture, counsel and encourage the continued robust growth and development of startups and existing small businesses 

In [22]:
results_df["extracted_tasks_flant5"][2]

'APPALACHIAN HIGHLANDS WOMEN’S BUSINESS CENTER Kingsport, TennesseeDepartment: Kingsport Office of Small Business Development & Entrepreneurship (KOSBE)Reports to: Chief Business Development Officer, Kingsport Chamber - KOSBEDirect Reports: AppH-WBC StaffType of Position: DirectorWork Schedule: Full-Time (40 hours per week). Exempt Status: ExemptLocation: This is not a remote position, this in office in Kingsport, Tenn. AppH-WBC is located in Kingsport, Tenn. Under the Kingsport Chamber and within the Kingsport Office of Small Business Development & Entrepreneurship (KOSBE). The office is within walking distance to downtown living, dining and shopping.ABOUT KOSBE:In 2004, the Kingsport Chamber and City of Kingsport jointly formed the Kingsport Office of Small Business Development & Entrepreneurship (KOSBE), to specifically nurture, counsel and encourage the continued robust growth and development of startups and existing small businesses in Kingsport, Tennessee. We are a technical assi

In [23]:
results_df["extracted_tasks_mistral"][2]

' - Provide strategic direction for training programs for women and minority entrepreneurs\n- Develop and deliver training and counseling programs\n- Plan and oversee the execution of conferences, seminars, and education and training events across service area\n- Establish an advisory council to support the vision of the program\n- Work with Kingsport Chamber leadership and staff in support of program operations and fiscal management\n- Oversee financial reporting, approve expenditures, and manage budget\n- Provide performance reports and statistical activities to SBA as required\n- Maintain an effective record-keeping and reporting system\n- Oversee the AppH-WBC client database\n- Manage marketing and publishing campaigns, newsletters, and marketing materials\n- Provide oversight of the AppH-WBC website content\n- Oversee performance of personnel, private consultants, and contractors\n- Establish a cohesive team and focus on staff development and long-term succession planning\n- Ident

### ✅ Summary:

As demonstrated above, **mistral model via Ollama** outperforms the **flan-t5-large model** in extracting job tasks:

* Mistral is **faster**, taking 4 minutes to process 10 job postings, compared to 6 minutes with flan-t5-large.
* Mistral handles **much longer inputs**, while flan-t5-large has a 512-token limit. This causes input truncation, potentially missing tasks listed toward the end of a posting.
* In terms of the **quality of "extracted_tasks"**, Mistral performs significantly better, especially for longer job postings (e.g. Job posting 2).

‼️ Therefore, I decided to use the **Mistral** model to extract job tasks from all job postings. ‼️


I became curious why Mistral via Ollama runs on my computer, while `HuggingFaceH4/zephyr-7b-beta` does not—despite both models having 7 billion parameters.
* The reason is that Mistral via Ollama runs better on my Mac because it uses a **quantized** model, loaded with a lightweight C++ engine optimized for Apple Silicon (M1/M2). It uses less RAM. 
* whereas models from Hugging Face often run in full precision using PyTorch, which is heavy and not optimized for Mac.

Below, I summarize the differences between Mistral and Flan-T5, with help from ChatGPT.


#### Overview

| Feature               | **Mistral**                            | **Flan-T5**                                   |
|-----------------------|----------------------------------------|-----------------------------------------------|
| **Type**              | Decoder-only (causal) transformer      | Encoder–decoder (seq2seq) transformer         |
| **Architecture**      | Like GPT, LLaMA                        | Like T5, BART                                 |
| **Pretraining Goal**  | Next-token prediction (causal LM)      | Masked span denoising + supervised finetune   |
| **Instruction-tuned?**| ✅ Yes (`Mistral-Instruct`)            | ✅ Yes (`flan-t5-large`)                       |
| **LLM?**              | ✅ Yes                                  | ✅ Yes                                         |

So **Mistral** and **Flan-T5** are both LLMs, but they belong to different model families.


#### Technical Differences

| Dimension             | **Mistral-7B**                         | **Flan-T5-Large**                              |
|-----------------------|----------------------------------------|------------------------------------------------|
| **Size (parameters)** | 7 billion                              | ~780 million                                   |
| **Architecture**      | Decoder-only (like GPT)                | Encoder-decoder (like T5)                      |
| **Token limit**       | 8,192 tokens                           | 512 tokens                                     |
| **Inference Style**   | Chat-style / Autoregressive            | Text-to-text generation                        |
| **Speed on M1 Mac**   | Fast (if quantized via Ollama)         | Slow (RAM-heavy unless small)                  |
| **Training**          | Trained from scratch (next-token)      | Finetuned on tasks/instructions                |
| **Best for**          | Chat, long text generation             | Structured tasks (e.g., QA, summarization)     |


#### Use Cases

| Task                            | **Better Model**        | **Why?**                                        |
|----------------------------------|--------------------------|--------------------------------------------------|
| Long job posting task extraction | ✅ Mistral               | Handles long input + fast inference              |
| Short-form classification task   | ✅ Flan-T5-Large         | Finetuned for instructions                       |
| Few-shot chat completion         | ✅ Mistral               | Chat-style model with long context               |
| Constrained output (e.g. JSON)   | ⚠️ Depends               | Flan-T5 better for structured formats            |
